In [9]:
import xarray as xr
from dea_tools.temporal import xr_phenology
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = xr.open_dataset('/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/LAI_mgap_90_w_7_v2.nc')

In [11]:
suffix = 'safra'

# the metrics
basic_pheno_stats = [
    "SOS","vSOS",
    "POS", "vPOS",
    "EOS", "vEOS",
    "Trough", "LOS","AOS",
    "ROG","ROS",
]
method_sos = "first"
method_eos = "last"

#### Trying the loop, weird error

In [22]:
dates = []
for year in np.unique(df.time.dt.year)[:-1]:
    da = df['LAI'].sel(time=slice(f'{year}-06-20',f'{year+1}-03-25'))
    da = da.compute()
    stats = xr_phenology( da, method_sos=method_sos, method_eos=method_eos, stats=basic_pheno_stats,
                    verbose=False )
    # add results to dict
    #datelabel = f'{year}-06-20'
    date = np.datetime64(f'{year}-06-20')
    dates.append(date)
    stats = stats.expand_dims(dim='time')
    stats.to_netcdf(f'/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_{year}_{suffix}.nc')
    del da, stats


/home/mad/geo/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/mad/geo/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/mad/geo/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/mad/geo/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/mad/geo/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/home/mad/geo/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:215: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
/hom

In [20]:
stats2 = stats.expand_dims(dim='time')

In [15]:
from glob import glob

In [26]:
files = sorted(glob('/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph*safra.nc'))
files

['/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2013_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2014_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2015_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2016_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2017_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2018_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2019_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2020_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2021_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/ph_2022_safra.nc',
 '/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-

In [29]:
DF = xr.open_mfdataset(files, concat_dim='time', combine='nested')

In [34]:
DF = DF.assign_coords({'time':dates})

/tmp/ipykernel_146889/689999748.py:1: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  DF = DF.assign_coords({'time':dates})


In [36]:
DF.to_netcdf('/mnt/geodata/Clientes/0FARMS/a5b9ae81-434a-4224-a81a-3c0ba44102b8/results/phenology_safra.nc')

In [33]:
dates

[numpy.datetime64('2013-06-20'),
 numpy.datetime64('2014-06-20'),
 numpy.datetime64('2015-06-20'),
 numpy.datetime64('2016-06-20'),
 numpy.datetime64('2017-06-20'),
 numpy.datetime64('2018-06-20'),
 numpy.datetime64('2019-06-20'),
 numpy.datetime64('2020-06-20'),
 numpy.datetime64('2021-06-20'),
 numpy.datetime64('2022-06-20'),
 numpy.datetime64('2023-06-20')]